# Sequence Tag

本笔记参看了[blog](https://www.cnblogs.com/jiangxinyang/p/9368482.html)

本笔记知识笼统地概括了序列标注的实现工具以及比较各种工具之间的优劣。

在深度学习发展起来之前，序列标注问题的解决方案都是借助于[HMM](../ml/hmm.ipynb)，最大熵模型，[CRF](../ml/crf.ipynb)模型。尤其是CRF，是解决序列标注问题的主流方法。随着深度学习的发展，[RNN](../dl/rnn.ipynb)在序列标注问题中取得了巨大的成果。而且深度学习中的end-to-end，也让序列标注问题变得更简单了。

序列标注问题包括[分词](word_segment.ipynb)，词性标注，命名实体识别，关键词抽取，词义角色标注等等。我们只要在做序列标注时给定特定的标签集合，就可以进行序列标注。

## BIO标注

工欲善其事, 必先利其器. 序列标注中的标注到底是个什么东西呢。

将每个元素标注为“B-X”、“I-X”或者“O”。其中，“B-X”表示此元素所在的片段属于X类型并且此元素在此片段的开头，“I-X”表示此元素所在的片段属于X类型并且此元素在此片段的中间位置，“O”表示不属于任何类型。

比如，我们将 X 表示为名词短语（Noun Phrase, NP），则BIO的三个标记为：

1. B-NP：名词短语的开头

2. I-NP：名词短语的中间

3. O：不是名词短语

我们可以进一步将BIO应用到NER中，来定义所有的命名实体（人名、组织名、地点、时间等），那么我们会有许多 B 和 I 的类别，如 B-PERS、I-PERS、B-ORG、I-ORG等。

## 几种方法的比较

- 循环神经网络家族：像RNN、LSTM、BILSTM这些模型，它们在序列建模上很强大，它们能够获取长远的上下文信息，此外还具备神经网络拟合非线性的能力，这些都是CRF无法超越的地方，对于$t$时刻来说，输出层$y_t$受到隐层$h_t$和输入层$x_t$的影响，但是$y_t$和其他时刻的$y_t$是相互独立的，对当前$t$时刻来说，我们希望找到一个概率最大的$y_t$，但其他时刻的$y_t$对当前$y_t$没有影响，如果$y_t$之间存在较强的依赖关系的话，模型无法对这些约束进行建模，模型的性能将受到限制。

- CRF：它不能考虑长远的上下文信息，它更多考虑的是整个句子的局部特征的线性加权组合(通过特征模版去扫描整个句子)。关键的一点是，CRF的模型为$p(y|x, \theta)$，注意这里$y$和$x$都是序列，优化的是一个序列$y = (y_1, y_2,\ldots,  y_n)$，而不是某个时刻的$y_t$，即找到一个概率最高的序列$y = (y_1, y_2, \ldots, y_n)$使得$p(y_1, y_2, \ldots, y_n| x, \theta)$最高，它计算的是一种联合概率，优化的是整个序列，而不是将每个时刻的最优拼接起来，在这一点上CRF要优于LSTM。

- HMM：CRF不管是在实践还是理论上都要优于HMM，HMM模型的参数主要是“初始的状态分布”，“状态之间的概率转移矩阵”，“状态到观测的概率转移矩阵”，这些信息在CRF中都可以有，例如：在特征模版中考虑$h(y_1), f(y_{i-1}, y_i), g(y_i, x_i)$等特征。

- CRF与LSTM：从数据规模来说，在数据规模较小时，CRF的试验效果要略优于BILSTM，当数据规模较大时，BILSTM的效果应该会超过CRF。从场景来说，如果需要识别的任务不需要太依赖长久的信息，此时RNN等模型只会增加额外的复杂度。
　　
- CNN＋BILSTM＋CRF：这是目前学术界比较流行的做法，BILSTM＋CRF是为了结合以上两个模型的优点，CNN主要是处理英文的情况，英文单词是由更细粒度的字母组成，这些字母潜藏着一些特征（例如：前缀后缀特征），通过CNN的卷积操作提取这些特征，在中文中可能并不适用（中文单字无法分解，除非是基于分词后），这里简单举一个例子，例如词性标注场景，单词football与basketball被标为名词的概率较高， 这里后缀ball就是类似这种特征。
